## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

In [2]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
from lonelyboy.geospatial import group_patterns_v2 as gsgp

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

In [3]:
import psycopg2
import psycopg2.extras
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point, LineString, shape
from haversine import haversine
from datetime import datetime, timedelta

In [4]:
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime

## Import Libraries for Visualizations

In [5]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
get_ipython().magic('matplotlib qt')

##  Importing the Server Credentials & Connectiing to Server and Fetch 48hrs of Trajectory Data

In [6]:
%%time
properties = configparser.ConfigParser()
properties.read(os.path.join('.','sql_server.ini'))
properties = properties['SERVER']

host    = properties['host']
db_name = properties['db_name']
uname   = properties['uname']
pw      = properties['pw']
port    = properties['port']

con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)

Wall time: 202 ms


## Fetching Data from Database using a Python List with:
  * ### PostGIS (GeoPandas)
  * ### PostgreSQL (Pandas)

In [6]:
mmsi_list = pd.read_pickle('data/pkl/mmsi_list.pckl')[0]

In [7]:
mmsis = tuple(np.random.choice(mmsi_list, 10))
mmsis

(538004561,
 356673000,
 228118700,
 238301000,
 241154000,
 477095600,
 538005385,
 338640000,
 210943000,
 227546430)

In [13]:
traj_sql = 'SELECT * FROM ais_data.dynamic_ships WHERE mmsi IN %s'
traj = gpd.GeoDataFrame.from_postgis(traj_sql%(mmsis,), con, geom_col='geom')

In [22]:
df = pd.read_sql_query('SELECT id,mmsi,turn,speed,course,heading,lon,lat,ts,traj_id, trip_id FROM ais_data.dynamic_ships_min_trip_card_3_segmented_12h WHERE mmsi = %s'%(371415000,),con=con)
df.head()

,id,mmsi,turn,speed,course,heading,lon,lat,ts,traj_id,trip_id
0,6933565,371415000,127.0,11.1,71.0,65,-5.378398,48.042583,1456916014,0,0
1,6933568,371415000,0.0,11.5,71.0,65,-5.363332,48.046017,1456916214,0,0
2,6933571,371415000,0.0,11.5,72.0,65,-5.351282,48.048534,1456916374,0,0
3,6933574,371415000,0.0,11.5,72.0,65,-5.345832,48.049633,1456916445,0,0
4,6933577,371415000,0.0,11.6,73.0,65,-5.345048,48.049767,1456916454,0,0


In [21]:
gsplt.map_plot(traj)

---

In [11]:
ports.head()

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POINT (-1.771798868659233 48.60274269672541)
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POINT (-2.001990119062326 48.48369993456267)
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POINT (-4.341204251638414 48.09709590770091)
3,4,port.101,101,Brézellec,29028,79105.4,2364190.0,POINT (-4.661115947908725 48.06804110561076)
4,5,port.102,102,Sein,29083,64562.5,2362180.0,POINT (-4.852944548180974 48.03825273921113)


In [12]:
traj.head()

,id,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,37866355,227546430,NaN,NaN,0.0,104.8,511,-4.432802,48.390810,1456413771,POINT (-4.4328017 48.39081)
1,37866382,227546430,NaN,NaN,0.0,0.0,511,-4.432793,48.390827,1456415031,POINT (-4.432793 48.390827)
2,37866401,227546430,NaN,NaN,0.0,221.2,511,-4.432800,48.390810,1456415570,POINT (-4.4328 48.39081)
3,37866423,227546430,NaN,NaN,0.0,0.0,511,-4.432797,48.390810,1456415932,POINT (-4.4327965 48.39081)
4,37866424,227546430,NaN,NaN,0.0,0.0,511,-4.432797,48.390810,1456415932,POINT (-4.4327965 48.39081)


## Trying to get data for selected mmsi using a generator from CSV file (fingers crossed)

In [8]:
for row in traj.itertuples():
    print (row)
    break

Pandas(Index=0, id=31329260, mmsi=227941000, status=7.0, turn=0.0, speed=0.0, course=285.0, heading=8, lon=-4.3272133, lat=48.100086, ts=1456802711, geom=<shapely.geometry.point.Point object at 0x7f28131725c0>)


In [58]:
query =  "INSERT INTO ais_data.dynamic_ships_resampled_1min (mmsi, status, turn, speed, velocity, course, heading, lon, lat, ts) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"
query.split('(')[1].

'mmsi, status, turn, speed, velocity, course, heading, lon, lat, ts) VALUES '

In [12]:
%%time
iter_csv = pd.read_csv('data/csv/nari_dynamic.csv', chunksize=50000)
df = pd.concat((chunk[chunk['mmsi'].isin([246090000])] for chunk in iter_csv), ignore_index=True)

CPU times: user 11.6 s, sys: 1.2 s, total: 12.8 s
Wall time: 12.8 s


In [16]:
df.head()

,mmsi,status,turn,speed,course,heading,lon,lat,ts
0,246090000,0.0,0.0,7.3,57.0,56,-5.205065,48.645767,1443790500
1,246090000,0.0,0.0,9.4,300.0,300,-4.669798,47.840084,1445243523
2,246090000,0.0,0.0,9.4,300.0,300,-4.692632,47.849316,1445243933
3,246090000,0.0,0.0,9.3,300.0,300,-4.697632,47.851300,1445244023
4,246090000,0.0,0.0,9.4,300.0,301,-4.698132,47.851500,1445244033


In [14]:
gspp.gdf_from_df(df, crs={'init':'epsg:4326'})

,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,538003354,0.0,0.0,12.4,58.0,59,-5.277512,48.946890,1443799994,POINT (-5.2775116 48.94689)
1,477967700,0.0,0.0,17.3,37.5,36,-5.756353,48.562070,1444758539,POINT (-5.756353400000001 48.56207)
2,538004252,0.0,0.0,12.3,58.2,58,-5.304182,48.902900,1445427824,POINT (-5.3041816 48.9029)
3,538003920,0.0,4.0,12.1,55.6,60,-5.569274,48.793060,1445906007,POINT (-5.5692735 48.79306)
4,538002755,0.0,0.0,13.4,27.2,24,-6.158875,48.114143,1446144200,POINT (-6.158875 48.114143)
5,538002959,0.0,127.0,7.0,218.0,233,-5.673482,48.954450,1449290610,POINT (-5.6734815 48.95445)


* #### Packaging to a Function

In [20]:
def read_csv_generator(file_path, chunksize=50000, sep=',', **kwargs):
    pd_iter = pd.read_csv(file_path, chunksize=chunksize, sep=sep, **kwargs)
    return pd_iter

for chunk in read_csv_generator('data/csv/nari_dynamic.csv', chunksize=50000):
    if len(chunk) != 0:
        print (chunk.head()) 
        break

        mmsi  status  turn  speed  course  heading       lon        lat  \
0  245257000       0     0    0.1    13.1       36 -4.465718  48.382490   
1  227705102      15  -127    0.0   262.7      511 -4.496571  48.382420   
2  228131600      15  -127    8.5   263.7      511 -4.644325  48.092247   
3  228051000       0  -127    0.0   295.0      511 -4.485108  48.381320   
4  227574020      15  -127    0.1   248.6      511 -4.495441  48.383660   

           ts  
0  1443650402  
1  1443650403  
2  1443650404  
3  1443650405  
4  1443650406  


## Trying to get data for selected mmsi using a generator from PostgreSQL (good luck with that)

In [ ]:
%%time

con = psycopg2.connect(database=db_name, user=uname, password=pw, host=host, port=port)
query = 'SELECT * FROM ais_data.dynamic_ships'
# gdf = pd.read_sql('SELECT * FROM ais_data.dynamic_ships', con=con)
size = 50000
i = 1
gdf = pd.DataFrame()
for chunk in pd.read_sql(query, con=con, chunksize=size):
    print (size*i)
    gdf = gdf.append(chunk.loc[chunk.mmsi == 227941000].drop('geom', axis=1))
    i+=1
    
con.close()

## RAM OVERLOAD (AGAIN!!!!)

## Sample Code from [PostgreSQL with Chunksize](http://jgardiner.co.uk/blog/read_sql_pandas)

In [ ]:
#Chunked access
start = time.time()
engine = create_engine(conn_str)
size = 10**4
df = pd.concat((x for x in pd.read_sql(iso_cmdb_base, engine, coerce_float=False, chunksize=size)),
               ignore_index=True)
print('time:', (time.time() - start)/60, 'minutes or ', time.time() - start, 'seconds')